In [4]:
import pandas as pd

# Load merged LLaMA3.2 output
df = pd.read_csv("LLaMA3.2_Merged_Multilingual.csv")

# Check if all three predictions agree
df["All_Agree"] = (
    (df["LLaMA_EN"] == df["LLaMA_FA"]) &
    (df["LLaMA_EN"] == df["LLaMA_IT"])
)

# Calculate agreement rate
agreement_rate = df["All_Agree"].mean() * 100
print(f"✅ Full agreement across EN, FA, IT: {agreement_rate:.2f}%")


✅ Full agreement across EN, FA, IT: 23.33%


In [5]:
df["Disagree"] = ~df["All_Agree"]
disagreements_by_domain = df[df["Disagree"]].groupby("Domain_EN").size()
print(disagreements_by_domain)


Domain_EN
Activism & Social Change    5
Arts & Culture              4
Business & Economics        2
Caregiving & Parenting      3
Education                   3
Healthcare                  4
Hospitality & Service       4
Journalism & Media          4
Law & Justice               5
Politics & Leadership       5
Religion & Spirituality     4
Science & Technology        3
dtype: int64


In [6]:
import pandas as pd

# Load the merged LLaMA 3.2 output
df = pd.read_csv("LLaMA3.2_Merged_Multilingual.csv")

# Add Disagreement_Type column
df["Disagreement_Type"] = df.apply(lambda row: 
    "FA diverged" if row["LLaMA_EN"] == row["LLaMA_IT"] != row["LLaMA_FA"] else
    "IT diverged" if row["LLaMA_EN"] == row["LLaMA_FA"] != row["LLaMA_IT"] else
    "EN diverged" if row["LLaMA_FA"] == row["LLaMA_IT"] != row["LLaMA_EN"] else
    "All different", axis=1
)

# Print breakdown
print(df["Disagreement_Type"].value_counts())


Disagreement_Type
All different    30
FA diverged      14
IT diverged      10
EN diverged       6
Name: count, dtype: int64


In [7]:
print("Option frequency in FA:")
print(df["LLaMA_FA"].value_counts(normalize=True).round(2) * 100)

Option frequency in FA:
LLaMA_FA
C    37.0
B    33.0
D    23.0
A     7.0
Name: proportion, dtype: float64


In [8]:
import pandas as pd

# Load your aligned LLaMA results
df = pd.read_csv("LLaMA3.2_Merged_Multilingual.csv")

# Identify disagreement rows (not all 3 agree)
df["Disagreement"] = (df["LLaMA_EN"] != df["LLaMA_FA"]) | \
                     (df["LLaMA_EN"] != df["LLaMA_IT"]) | \
                     (df["LLaMA_FA"] != df["LLaMA_IT"])

# Export only the rows with disagreement
df_disagreement = df[df["Disagreement"] == True]
df_disagreement.to_csv("LLaMA3.2_Disagreements.csv", index=False)
print(f"✅ Exported {len(df_disagreement)} disagreement cases to LLaMA3.2_Disagreements.csv")

# Count disagreement cases per domain
disagreement_by_domain = df_disagreement["Domain_EN"].value_counts().reset_index()
disagreement_by_domain.columns = ["Domain", "Disagreement_Count"]
disagreement_by_domain.to_csv("LLaMA3.2_Disagreement_By_Domain.csv", index=False)
print("✅ Saved domain-wise disagreement counts to LLaMA3.2_Disagreement_By_Domain.csv")

# (Optional) Show most frequent disagreement domains in console
print("\n📊 Top domains with most disagreement:")
print(disagreement_by_domain.head(10))


✅ Exported 46 disagreement cases to LLaMA3.2_Disagreements.csv
✅ Saved domain-wise disagreement counts to LLaMA3.2_Disagreement_By_Domain.csv

📊 Top domains with most disagreement:
                     Domain  Disagreement_Count
0     Politics & Leadership                   5
1             Law & Justice                   5
2  Activism & Social Change                   5
3                Healthcare                   4
4     Hospitality & Service                   4
5   Religion & Spirituality                   4
6        Journalism & Media                   4
7            Arts & Culture                   4
8      Science & Technology                   3
9                 Education                   3


In [9]:
import pandas as pd
from collections import Counter

# Load merged results file for LLaMA 3.2
df = pd.read_csv("LLaMA3.2_Merged_Multilingual.csv")

# Check full agreement
df["Full_Agreement"] = (df["LLaMA_EN"] == df["LLaMA_FA"]) & (df["LLaMA_EN"] == df["LLaMA_IT"])

# Classify type of disagreement
def classify_disagreement(row):
    choices = {row["LLaMA_EN"], row["LLaMA_FA"], row["LLaMA_IT"]}
    if len(choices) == 1:
        return "All same"
    elif len(choices) == 3:
        return "All different"
    elif row["LLaMA_EN"] != row["LLaMA_FA"] and row["LLaMA_EN"] == row["LLaMA_IT"]:
        return "FA diverged"
    elif row["LLaMA_EN"] != row["LLaMA_IT"] and row["LLaMA_EN"] == row["LLaMA_FA"]:
        return "IT diverged"
    elif row["LLaMA_EN"] != row["LLaMA_FA"] and row["LLaMA_FA"] == row["LLaMA_IT"]:
        return "EN diverged"
    else:
        return "Other"

df["Disagreement_Type"] = df.apply(classify_disagreement, axis=1)

# Count full agreement
agreement_count = df["Full_Agreement"].sum()
total = len(df)
agreement_pct = agreement_count / total * 100

# Disagreement breakdown
disagreement_summary = df["Disagreement_Type"].value_counts().to_frame("count")

# Frequency of choices in Persian
fa_freq = df["LLaMA_FA"].value_counts(normalize=True).rename("proportion").to_frame()

# Domain-wise disagreement
domain_disagreements = df[df["Disagreement_Type"] != "All same"]["Domain_EN"].value_counts().to_frame("count")

# Display everything
print("🔎 Full Agreement Rate:")
print(f"{agreement_count}/{total} prompts ({agreement_pct:.2f}%)\n")

print("🧩 Disagreement Types:")
print(disagreement_summary, "\n")

print("🗳️ Option Frequency in Persian (LLaMA_FA):")
print(fa_freq, "\n")

print("🌍 Domain-wise Disagreements:")
print(domain_disagreements)


🔎 Full Agreement Rate:
14/60 prompts (23.33%)

🧩 Disagreement Types:
                   count
Disagreement_Type       
All different         16
FA diverged           14
All same              14
IT diverged           10
EN diverged            6 

🗳️ Option Frequency in Persian (LLaMA_FA):
          proportion
LLaMA_FA            
C           0.366667
B           0.333333
D           0.233333
A           0.066667 

🌍 Domain-wise Disagreements:
                          count
Domain_EN                      
Politics & Leadership         5
Law & Justice                 5
Activism & Social Change      5
Healthcare                    4
Hospitality & Service         4
Religion & Spirituality       4
Journalism & Media            4
Arts & Culture                4
Science & Technology          3
Education                     3
Caregiving & Parenting        3
Business & Economics          2


In [1]:
import pandas as pd
import numpy as np

def bootstrap_llama32_agreement_CI(filepath, prefix="LLaMA", num_bootstrap=1000, confidence=0.95):
    df = pd.read_csv(filepath)
    en, fa, it = f"{prefix}_EN", f"{prefix}_FA", f"{prefix}_IT"
    df["Full_Agreement"] = (df[en] == df[fa]) & (df[en] == df[it])

    bootstrapped_agreements = []
    for _ in range(num_bootstrap):
        sample = df.sample(frac=1.0, replace=True)
        agreement_pct = sample["Full_Agreement"].mean() * 100
        bootstrapped_agreements.append(agreement_pct)

    lower_bound = np.percentile(bootstrapped_agreements, (1 - confidence) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_agreements, (1 + confidence) / 2 * 100)
    mean_agreement = np.mean(bootstrapped_agreements)

    print(f"Bootstrapped {int(confidence*100)}% CI for LLaMA 3.2 agreement:")
    print(f"Mean agreement: {mean_agreement:.2f}%")
    print(f"{int(confidence*100)}% Confidence Interval: [{lower_bound:.2f}%, {upper_bound:.2f}%]")

# Example usage:
bootstrap_llama32_agreement_CI("LLaMA3.2_Merged_Multilingual.csv")


Bootstrapped 95% CI for LLaMA 3.2 agreement:
Mean agreement: 23.27%
95% Confidence Interval: [13.33%, 35.00%]


In [4]:
import pandas as pd
import numpy as np

def bootstrap_llama32_disagreement_CI(filepath, prefix="LLaMA", num_bootstrap=1000, confidence=0.95):
    df = pd.read_csv(filepath)
    en, fa, it = f"{prefix}_EN", f"{prefix}_FA", f"{prefix}_IT"

    # Disagreement: at least one language differs
    df["Disagreement"] = (df[en] != df[fa]) | (df[en] != df[it]) | (df[fa] != df[it])

    bootstrapped_disagreements = []
    for _ in range(num_bootstrap):
        sample = df.sample(frac=1.0, replace=True)
        disagreement_pct = sample["Disagreement"].mean() * 100
        bootstrapped_disagreements.append(disagreement_pct)

    lower_bound = np.percentile(bootstrapped_disagreements, (1 - confidence) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_disagreements, (1 + confidence) / 2 * 100)
    mean_disagreement = np.mean(bootstrapped_disagreements)

    print(f"Bootstrapped {int(confidence*100)}% CI for LLaMA 3.2 disagreement:")
    print(f"Mean disagreement: {mean_disagreement:.2f}%")
    print(f"{int(confidence*100)}% Confidence Interval: [{lower_bound:.2f}%, {upper_bound:.2f}%]")

# Example usage:
bootstrap_llama32_disagreement_CI("LLaMA3.2_Merged_Multilingual.csv")


Bootstrapped 95% CI for LLaMA 3.2 disagreement:
Mean disagreement: 76.83%
95% Confidence Interval: [65.00%, 88.33%]
